In [ ]:
import math
import cv2
import mediapipe as mp

# Angle calculator
def calculate_angle(a, b, c):
    a = [a.x, a.y]
    b = [b.x, b.y]
    c = [c.x, c.y]

    ba = [a[0] - b[0], a[1] - b[1]]
    bc = [c[0] - b[0], c[1] - b[1]]

    cosine_angle = (ba[0]*bc[0] + ba[1]*bc[1]) / (
        math.sqrt(ba[0]**2 + ba[1]**2) * math.sqrt(bc[0]**2 + bc[1]**2)
    )
    angle = math.degrees(math.acos(cosine_angle))
    return angle

# MediaPipe setup
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose()

cap = cv2.VideoCapture("annotated_person01_running_d1_uncomp.avi")

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        lm = results.pose_landmarks.landmark

        # LEFT LEG
        left_hip = lm[mp_pose.PoseLandmark.LEFT_HIP]
        left_knee = lm[mp_pose.PoseLandmark.LEFT_KNEE]
        left_ankle = lm[mp_pose.PoseLandmark.LEFT_ANKLE]
        left_foot = lm[mp_pose.PoseLandmark.LEFT_FOOT_INDEX]

        left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
        left_ankle_angle = calculate_angle(left_knee, left_ankle, left_foot)

        print(f"Left knee angle: {left_knee_angle:.2f}")
        print(f"Left ankle angle: {left_ankle_angle:.2f}")

        # Optional: draw on image
        frame_h, frame_w = frame.shape[:2]
        cv2.putText(frame, f"Knee: {int(left_knee_angle)}", 
                    (int(left_knee.x * frame_w), int(left_knee.y * frame_h)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
        cv2.putText(frame, f"Ankle: {int(left_ankle_angle)}", 
                    (int(left_ankle.x * frame_w), int(left_ankle.y * frame_h)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    cv2.imshow("Lower Body Angle Detection", frame)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
